# Conjunto de dados

COMPAS (Perfil de gerenciamento de infratores correcionais para sanções alternativas) é um algoritmo comercial popular usado por juízes e oficiais de liberdade condicional para avaliar a probabilidade de reincidência (reincidência) do réu criminal. Foi demonstrado que o algoritmo é tendencioso a favor dos arguidos brancos e contra os reclusos negros, com base num estudo de acompanhamento de 2 anos (ou seja, quem realmente cometeu crimes ou crimes violentos após 2 anos). O padrão de erros, medido pela precisão/sensibilidade, é notável.

# Processamento dos dados

In [ ]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 10.3 MB/s eta 0:00:00


In [ ]:
pip install dalex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for dalex: filename=dalex-1.7.2-py3-none-any.whl size=1042898 sha256=02d18b02e8b025f383470d814abe20ce6646b05f06da15318c0020c2c1813468
  Stored in directory: /root/.cache/pip/wheels/cd/5e/ec/848120d5b913158656c0f8ef316233dd3356effa64aef19fba
Successfully built dalex


In [ ]:
!pip install aif360 numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.7/259.7 kB 7.1 MB/s eta 0:00:00


In [ ]:
pip install --upgrade scikit-learn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 128.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sklearn-compat 0.1.3 requires scikit-learn<1.7,>=1.2, but you have scikit-learn 1.7.0 which is incompatible.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import plotly.express as px

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import RidgeClassifier, SGDClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score, f1_score
from sklearn import metrics

from io import StringIO
import sys

import re
import sys
from io import StringIO

import dalex as dx

from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_curve, auc


In [ ]:
compas = pd.read_csv("https://raw.githubusercontent.com/propublica/compas-analysis/master/compas-scores-two-years.csv")

In [ ]:
compas = compas[["sex", "age", "age_cat", "race", "juv_fel_count", "juv_misd_count",
                 "juv_other_count", "priors_count", "c_charge_degree",
                 "is_violent_recid", "two_year_recid"]]

In [ ]:
age_cat = compas.age_cat

In [ ]:
a = compas.sex.value_counts()

In [ ]:
compas.shape

(7214, 11)

In [ ]:
a

,count
sex,
Male,5819
Female,1395


In [ ]:
import plotly.express as px
fig = px.bar(a,text_auto=True)
fig.show()

In [ ]:
compas["sex"].replace({'Male': 1, 'Female': 0}, inplace=True)

/tmp/ipython-input-13-3408498363.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipython-input-13-3408498363.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [ ]:
compas=compas.drop(["race"], axis=1)

# Primeira etapa do experimento

* dataset do compas
* vários algoritmos
* várias métricas de **fairness**

Fairness by unawareness ou justiça através do desconhecimento é um conceito ingênuo de  justiça em que um algoritmo justo é definido quando não usa nenhum atributo protegido no processo de treinamento. Então não existe essa ideia de treinar sem o atributo protegido.

In [ ]:
compas = compas.drop("age_cat", axis =1)

In [ ]:
compas["sex"].replace({1: 'Male', 0:'Female'}, inplace=True)

/tmp/ipython-input-16-3655931398.py:1: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [ ]:
compas["sex"].value_counts()

,count
sex,
Male,5819
Female,1395


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(compas.drop(["two_year_recid"], axis =1),
                                                    compas.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
categorical_features = ['sex']
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

numerical_features = ["age", "priors_count"]
numerical_transformer = Pipeline(steps=[
    ('scale', StandardScaler())
])

preprocessor = ColumnTransformer(transformers=[
        ('cat', categorical_transformer, categorical_features),
        ('num', numerical_transformer, numerical_features)
])

### Treinando para vários algoritmos

In [ ]:
  clf_tree = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', DecisionTreeClassifier(max_depth=7, random_state=123))
  ])

  clf_forest = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RandomForestClassifier(n_estimators=200, max_depth=7, random_state=123))
  ])

  clf_logreg = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LogisticRegression())
  ])

  clf_gb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GradientBoostingClassifier(random_state=123))
  ])

  clf_xgb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', XGBClassifier(random_state=123))
  ])

  clf_lgbm = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', LGBMClassifier(random_state=123))
  ])

  clf_catboost = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', CatBoostClassifier(random_state=123, verbose=0))
  ])

  clf_ridge = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', RidgeClassifier())
  ])

  clf_sgd = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SGDClassifier(random_state=123))
  ])

  clf_svc = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', SVC(random_state=123))
  ])

  clf_knn = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', KNeighborsClassifier())
  ])

  clf_nb = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', GaussianNB())
  ])

  clf_mlp = Pipeline(steps=[
      ('preprocessor', preprocessor),
      ('classifier', MLPClassifier(random_state=123))
  ])

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    'SVC': clf_svc,
    'KNeighbors': clf_knn,
    'GaussianNB': clf_nb,
    'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


### Avaliando para várias métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)
    relatorio = {'accuracy': accuracy_score(y_test, y_pred), 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy                model
0   0.660508        Random Forest
1   0.671594  Logistic Regression
2   0.667898    Gradient Boosting
3   0.653580              XGBoost
4   0.667436             CatBoost
5   0.666513     Ridge Classifier
6   0.650346       SGD Classifier
7   0.677136                  SVC
8   0.621709           KNeighbors
9   0.653118           GaussianNB
10  0.676674        MLPClassifier


In [ ]:
resultados = []
for model_name, model in trained_models.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

df_resultados = pd.DataFrame(resultados)
print(df_resultados)

    accuracy    recall  precision       auc        f1                model
0   0.660508  0.544225   0.637805  0.648774  0.587310        Random Forest
1   0.671594  0.518210   0.667560  0.656115  0.583480  Logistic Regression
2   0.667898  0.540062   0.652010  0.654998  0.590780    Gradient Boosting
3   0.653580  0.552549   0.623972  0.643384  0.586093              XGBoost
4   0.667436  0.544225   0.649689  0.655003  0.592299             CatBoost
5   0.666513  0.499480   0.665742  0.649657  0.570749     Ridge Classifier
6   0.650346  0.629553   0.601392  0.648248  0.615150       SGD Classifier
7   0.677136  0.540062   0.668814  0.663304  0.597582                  SVC
8   0.621709  0.547347   0.578022  0.614205  0.562266           KNeighbors
9   0.653118  0.427680   0.671569  0.630368  0.522568           GaussianNB
10  0.676674  0.556712   0.661310  0.664569  0.604520        MLPClassifier


In [ ]:
#df_resultados.to_excel(r"Performance padrão dos algoritmos sem balanceamento.xlsx", index=False)

### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers = {}

for model_name, model in trained_models.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 8 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0435, mean = 0.443, max = 0.984
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.928, mean = 0.00094, max = 0.953
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

Para automatizar o processo de concatenar os resultados de desempenho de todos os modelos explicados com DALEX, irei iterar sobre os explicadores armazenados em um dicionário e concatenar seus resultados.

In [ ]:
performance_results = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name
    performance_results.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results = pd.concat(performance_results, ignore_index=True)

o método model_perfomance realiza o cálculo de medidas de desempenho do modelo


In [ ]:
protected = X_test.sex

In [ ]:
model_fairness_results = {}

for model_name, explainer in explainers.items():
    fairness_result = explainer.model_fairness(protected=protected, privileged="Female")
    model_fairness_results[model_name] = fairness_result

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
fairness_checks = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result in model_fairness_results.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks = pd.DataFrame(fairness_checks)

# Exibir o DataFrame com os resultados
print(all_fairness_checks)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Female'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
           TPR       ACC       PPV       FPR       STP
Male  1.580822  0.936782  1.004724  2.398305  2.024155
Métricas de viés detectadas para Random Forest: TPR=1.580822, ACC=0.936782, PPV=1.004724, FPR=2.398305, STP=2.024155
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on 'Female'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR

Este código captura e processa a saída do método fairness_check() para cada modelo, extrai as métricas relevantes e armazena essas informações em um DataFrame para análise. Isso facilita a visualização e comparação dos resultados de diferentes modelos em termos de métricas de viés detectadas.

In [ ]:
all_fairness_checks.head()

,Modelo,Num Biases Detected,TPR,ACC,PPV,FPR,STP
0,Decision Tree,3,1.482143,0.923186,0.982919,2.385246,1.940639
1,Random Forest,3,1.580822,0.936782,1.004724,2.398305,2.024155
2,Logistic Regression,3,2.268000,0.936441,0.819652,7.470588,3.562500
3,Gradient Boosting,3,1.635328,0.929379,0.948830,2.956044,2.216216
4,XGBoost,3,1.608219,0.929293,0.993631,2.508197,2.086124


In [ ]:
for model_name, fairness_result in model_fairness_results.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data = data.stack(dropna=False)


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



# Segunda etapa do experimento - Balancear os dados para mitigar o viés

* dataset do compas
* balanceamento dos dados no treinamento
* várias técnicas de balanceamento dos dados
* vários algoritmos
* várias métricas de **fairness**

In [ ]:
compas["sex"].replace({'Male': 1, 'Female': 0}, inplace=True)

/tmp/ipython-input-32-3408498363.py:1: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



### Abordagem 1 - Embaralhar a coluna de sexo desbalanceada

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(compas.drop(["two_year_recid"], axis =1),
                                                    compas.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
X_train_abordagemUm = X_train.copy().reset_index(drop=True)

X_train_abordagemUm["sex"] = X_train["sex"].sample(frac=1).reset_index(drop=True)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_abordagemUm, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemUm = pd.DataFrame(resultados)
print(df_resultados_abordagemUm)


    accuracy    recall  precision       auc        f1                model
0   0.685912  0.589770   0.663146  0.675995  0.624309        Random Forest
1   0.684527  0.524008   0.688615  0.667969  0.595139  Logistic Regression
2   0.687298  0.603340   0.660571  0.678638  0.630660    Gradient Boosting
3   0.684527  0.585595   0.662338  0.674322  0.621607              XGBoost
4   0.685912  0.579332   0.667067  0.674919  0.620112             CatBoost
5   0.685450  0.516701   0.694250  0.668044  0.592460     Ridge Classifier
6   0.656351  0.784969   0.582946  0.669618  0.669039       SGD Classifier
7   0.687760  0.555324   0.680307  0.674099  0.611494                  SVC
8   0.642494  0.556367   0.604308  0.633610  0.579348           KNeighbors
9   0.641109  0.303758   0.725686  0.606311  0.428256           GaussianNB
10  0.686374  0.590814   0.663540  0.676517  0.625069        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 8 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0789, mean = 0.453, max = 0.986
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.948, mean = -0.0107, max = 0.911
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV       FPR       STP
1  1.40411  0.975714  1.144558  1.598765  1.649805
Métricas de viés detectadas para Random Forest: TPR=1.40411, ACC=0.975714, PPV=1.144558, FPR=1.598765, STP=1.649805
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 2 - Duplicar os dados de treinamento com a classe oposta

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(compas.drop(["two_year_recid"], axis =1),
                                                    compas.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=123)

In [ ]:
df2 = X_train.copy()

df2["sex"] = df2["sex"].replace({"Male": "Female", "Female": "Male"})
X_train2 = pd.concat([df2, X_train])
print(X_train2.shape, X_train.shape)
X_train2['sex'].value_counts()

(10098, 8) (5049, 8)


,count
sex,
1,8130
0,1968


In [ ]:
y_train2 = np.concatenate([y_train, y_train])

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados duplicados no treinamento com o sexo (classe) oposto
trained_models_duplicados = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train2, y_train2)
    trained_models_duplicados[model_name] = model
    print(f"Treinando {model_name} com dados de treinamento duplicados")

Treinando Random Forest com dados de treinamento duplicados
Treinando Logistic Regression com dados de treinamento duplicados
Treinando Gradient Boosting com dados de treinamento duplicados
Treinando XGBoost com dados de treinamento duplicados
Treinando CatBoost com dados de treinamento duplicados
Treinando Ridge Classifier com dados de treinamento duplicados
Treinando SGD Classifier com dados de treinamento duplicados
Treinando SVC com dados de treinamento duplicados
Treinando KNeighbors com dados de treinamento duplicados
Treinando GaussianNB com dados de treinamento duplicados
Treinando MLPClassifier com dados de treinamento duplicados


#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_duplicados.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemDois = pd.DataFrame(resultados)
print(df_resultados_abordagemDois)


    accuracy    recall  precision       auc        f1                model
0   0.666051  0.558793   0.642344  0.655227  0.597663        Random Forest
1   0.671594  0.518210   0.667560  0.656115  0.583480  Logistic Regression
2   0.667898  0.540062   0.652010  0.654998  0.590780    Gradient Boosting
3   0.650808  0.535900   0.624242  0.639213  0.576708              XGBoost
4   0.665127  0.543184   0.646040  0.652821  0.590164             CatBoost
5   0.666513  0.499480   0.665742  0.649657  0.570749     Ridge Classifier
6   0.668360  0.569199   0.642773  0.658354  0.603753       SGD Classifier
7   0.678522  0.533819   0.674113  0.663919  0.595819                  SVC
8   0.633256  0.566077   0.590662  0.626477  0.578108           KNeighbors
9   0.653118  0.427680   0.671569  0.630368  0.522568           GaussianNB
10  0.675751  0.551509   0.661673  0.663213  0.601589        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_dois = {}

for model_name, model in trained_models_duplicados.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_dois[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 8 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0408, mean = 0.443, max = 0.992
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.926, mean = 0.000728, max = 0.959
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento_dois = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_dois.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_dois.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_dois = pd.concat(performance_results_balanceamento_dois, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_dois = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_dois.items():
    fairness_result_balanceamento_dois = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_dois[model_name] = fairness_result_balanceamento_dois

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_dois = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_dois is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_dois.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_dois.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_dois = pd.DataFrame(fairness_checks_balanceamento_dois)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_dois)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.532468  0.938659  1.004687  2.327869  1.963134
Métricas de viés detectadas para Random Forest: TPR=1.532468, ACC=0.938659, PPV=1.004687, FPR=2.327869, STP=1.963134
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
     TPR       ACC       PP

In [ ]:
# Gráfico de Barras: Comparação do Número de Métricas de Viés Detectadas por Modelo
fig = px.bar(
    all_fairness_checks_balanceamento_dois,
    x='Modelo',
    y='Num Biases Detected',
    title="Número de Métricas de Viés Detectadas por Modelo",
    color='Num Biases Detected',  # Colore as barras com base no valor
    color_continuous_scale='viridis',
    labels={"Num Biases Detected": "Número de Métricas de Viés", "Modelo": "Modelo"}
)

# Ajuste de layout para legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    title_x=0.5,  # Centraliza o título
    yaxis_title="Número de Métricas de Viés",
    xaxis_title="Modelo"
)

# Exibindo o gráfico
fig.show()

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_dois.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_dois in model_fairness_results_balanceamento_dois.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_dois.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 3 - Subamostragem da Classe Majoritária

In [ ]:
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
from sklearn.model_selection import train_test_split

# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas.drop(["two_year_recid"], axis=1),
                                                    compas.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

# Aplicar subamostragem da classe majoritária
rus = RandomUnderSampler(random_state=42)
X_train_res, y_train_res = rus.fit_resample(X_train, y_train)

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_subamostragem = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_res, y_train_res)
    trained_models_subamostragem[model_name] = model
    print(f"Treinado {model_name} com subamostragem")

Treinado Random Forest com subamostragem
Treinado Logistic Regression com subamostragem
Treinado Gradient Boosting com subamostragem
Treinado XGBoost com subamostragem
Treinado CatBoost com subamostragem
Treinado Ridge Classifier com subamostragem
Treinado SGD Classifier com subamostragem
Treinado SVC com subamostragem
Treinado KNeighbors com subamostragem
Treinado GaussianNB com subamostragem
Treinado MLPClassifier com subamostragem


#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_subamostragem.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemTres = pd.DataFrame(resultados)
print(df_resultados_abordagemTres)


    accuracy    recall  precision       auc        f1                model
0   0.686836  0.647182   0.645833  0.682746  0.646507        Random Forest
1   0.679446  0.705637   0.621324  0.682147  0.660802  Logistic Regression
2   0.692841  0.659708   0.650875  0.689423  0.655262    Gradient Boosting
3   0.678522  0.656576   0.631526  0.676258  0.643808              XGBoost
4   0.691455  0.675365   0.644422  0.689795  0.659531             CatBoost
5   0.674827  0.706681   0.615455  0.678112  0.657920     Ridge Classifier
6   0.678522  0.487474   0.694940  0.658816  0.573006       SGD Classifier
7   0.684065  0.672234   0.635108  0.682844  0.653144                  SVC
8   0.638799  0.375783   0.661765  0.611669  0.479361           KNeighbors
9   0.620785  0.727557   0.554495  0.631799  0.629345           GaussianNB
10  0.694226  0.681628   0.646535  0.692927  0.663618        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_tres = {}

for model_name, model in trained_models_subamostragem.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_tres[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 8 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0422, mean = 0.496, max = 0.992
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.934, mean = -0.053, max = 0.871
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento_tres = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_tres.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_tres.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_tres = pd.concat(performance_results_balanceamento_tres, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_tres = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_tres.items():
    fairness_result_balanceamento_tres = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_tres[model_name] = fairness_result_balanceamento_tres

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_tres = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_tres is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_tres.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_tres.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_tres = pd.DataFrame(fairness_checks_balanceamento_tres)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_tres)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.449679  0.979943  1.135182  1.707182  1.714286
Métricas de viés detectadas para Random Forest: TPR=1.449679, ACC=0.979943, PPV=1.135182, FPR=1.707182, STP=1.714286
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC      

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_tres.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_tres in model_fairness_results_balanceamento_tres.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_tres.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 4 - Utilização da biblioteca SMOTE para criar exemplos sintéticos da classe minoritária

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
Categorical=compas.select_dtypes(include='object').columns.tolist()
Categorical

['c_charge_degree']

In [ ]:
# Supomos que 'compas' seja seu DataFrame original
categorical_columns = ['c_charge_degree']
non_categorical_columns = compas.drop(columns=categorical_columns)  # Selecionar as colunas restantes

# Inicializar o OneHotEncoder
encoder = OneHotEncoder(handle_unknown='ignore')

# Aplicar o encoder nas colunas categóricas
encoded_data = encoder.fit_transform(compas[categorical_columns]).toarray()  # Convertendo para array denso

# Criar um DataFrame com as colunas codificadas
encoded_df = pd.DataFrame(encoded_data,
                          columns=encoder.get_feature_names_out(categorical_columns),
                          index=compas.index)  # Preserva os índices originais

# Concatenar o DataFrame codificado com as colunas não categóricas
df_encoded = pd.concat([non_categorical_columns, encoded_df], axis=1)

# Exibir o DataFrame final
df_encoded.head()

,sex,age,juv_fel_count,juv_misd_count,juv_other_count,priors_count,is_violent_recid,two_year_recid,c_charge_degree_F,c_charge_degree_M
0,1,69,0,0,0,0,0,0,1.0,0.0
1,1,34,0,0,0,0,1,1,1.0,0.0
2,1,24,0,0,1,4,0,1,1.0,0.0
3,1,23,0,1,0,1,0,0,1.0,0.0
4,1,43,0,0,0,2,0,0,1.0,0.0


In [ ]:
compas_encoded = df_encoded

In [ ]:


# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)


# Aplicar SMOTE
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# Verificar o novo balanceamento
print(X_train_smote['sex'].value_counts())
print(y_train_smote.value_counts())


sex
1    4423
0    1089
Name: count, dtype: int64
two_year_recid
1    2756
0    2756
Name: count, dtype: int64


O SMOTE cria exemplos sintéticos da classe minoritária, aumentando o conjunto de dados de forma mais natural.

#### Treinando para vários algoritmos

In [ ]:
# Treinar os modelos com os dados subamostrados
trained_models_smote = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_smote, y_train_smote)
    trained_models_smote[model_name] = model
    print(f"Treinando {model_name} com smote")

Treinando Random Forest com smote
Treinando Logistic Regression com smote
Treinando Gradient Boosting com smote
Treinando XGBoost com smote
Treinando CatBoost com smote
Treinando Ridge Classifier com smote
Treinando SGD Classifier com smote
Treinando SVC com smote
Treinando KNeighbors com smote
Treinando GaussianNB com smote
Treinando MLPClassifier com smote


#### Avaliando para métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_smote.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemQuatro = pd.DataFrame(resultados)
print(df_resultados_abordagemQuatro)


    accuracy    recall  precision       auc        f1                model
0   0.687298  0.641962   0.648051  0.682622  0.644992        Random Forest
1   0.684065  0.705637   0.627087  0.686290  0.664047  Logistic Regression
2   0.691917  0.658664   0.649846  0.688487  0.654225    Gradient Boosting
3   0.676674  0.653445   0.629779  0.674278  0.641393              XGBoost
4   0.690531  0.661795   0.646939  0.687567  0.654283             CatBoost
5   0.681293  0.708768   0.622936  0.684127  0.663086     Ridge Classifier
6   0.680370  0.676409   0.629126  0.679961  0.651911       SGD Classifier
7   0.686374  0.681628   0.635833  0.685885  0.657935                  SVC
8   0.629561  0.606472   0.577535  0.627180  0.591650           KNeighbors
9   0.644342  0.708768   0.580342  0.650987  0.638158           GaussianNB
10  0.695612  0.696242   0.644444  0.695677  0.669343        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento_quatro = {}

for model_name, model in trained_models_smote.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento_quatro[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0567, mean = 0.493, max = 0.989
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.931, mean = -0.0504, max = 0.849
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento_quatro = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento_quatro.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento_quatro.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento_quatro = pd.concat(performance_results_balanceamento_quatro, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento_quatro = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento_quatro.items():
    fairness_result_balanceamento_quatro = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento_quatro[model_name] = fairness_result_balanceamento_quatro

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento_quatro = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento_quatro is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento_quatro.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento_quatro.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento_quatro = pd.DataFrame(fairness_checks_balanceamento_quatro)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento_quatro)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR   STP
1  1.516854  0.995652  1.166372  1.674033  1.75
Métricas de viés detectadas para Random Forest: TPR=1.516854, ACC=0.995652, PPV=1.166372, FPR=1.674033, STP=1.75
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       F

 graficos das metricas de fairness do dalex e avaliação para algunos modelos

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento_quatro.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
for model_name, fairness_result_balanceamento_quatro in model_fairness_results_balanceamento_quatro.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result_balanceamento_quatro.plot()
    plt.show()


Gerando gráfico para Decision Tree


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 5 - FairSmote (Implementação de um algoritmo que remove rótulos tendenciosos e rebalanceia as distribuições internas de forma que, com base no atributo sensível, os exemplos sejam iguais tanto nas classes positivas quanto negativas.)

In [ ]:
from __future__ import print_function, division
import pdb
import unittest
import random
from collections import Counter
from sklearn.neighbors import NearestNeighbors as NN

class smote(object):
  def __init__(self, pd_data, neighbor=5,r=2 ,up_to_num=[],auto=True):
    """
    :param pd_data: panda.DataFrame, the last column must be class label
    :param neighbor: num of nearst neighbors to select
    :param up_to_num: size of minorities to over-sampling
    :param up_to_max: if up_to_num is not supplied, all minority classes will
                      be over-sampled as much as majority class
    :return panda.DataFrame smoted data
    """
    self.set_data(pd_data)
    self.auto = auto
    self.neighbor = neighbor
    self.up_to_max = False
    self.up_to_num = up_to_num
    self.r = r
    self.label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #if up_to_num:
    #  label_num = len(set(pd_data[pd_data.columns[-1]].values))
    #  if label_num - 1 != len(up_to_num):
    #    raise ValueError(
    #      "should set smoted size for " + str(label_num - 1) + " minorities")
    #  self.up_to_num = up_to_num
    #else:
    #  self.up_to_max = True

  def set_data(self, pd_data):
    if not pd_data.empty:# and isinstance(
        #pd_data.ix[:, pd_data.columns[-1]].values[0], str):
      self.data = pd_data
    else:
      raise ValueError(
        "The last column of pd_data should be string as class label")

  def get_majority_num(self):
    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    return majority_num

  def run(self):
    """
    run smote
    """

    def get_ngbr(data_no_label, knn):
      rand_sample_idx = random.randint(0, len(data_no_label) - 1)
      rand_sample = data_no_label[rand_sample_idx]
      distance, ngbr = knn.kneighbors(rand_sample.reshape(1, -1))
      # while True:
      rand_ngbr_idx = random.randint(0, len(ngbr))
      #   if distance[rand_ngbr_idx] == 0:
      #     continue  # avoid the sample itself, where distance ==0
      #   else:
      return data_no_label[rand_ngbr_idx], rand_sample

    total_data = self.data.values.tolist()
    labelCont = Counter(self.data[self.data.columns[-1]].values)
    majority_num = max(labelCont.values())
    for label, num in labelCont.items():
      if num < majority_num:
        to_add = majority_num - num
        last_column = self.data[self.data.columns[-1]]
        data_w_label = self.data.loc[last_column == label]
        data_no_label = data_w_label[self.data.columns[:-1]].values
        if len(data_no_label) < self.neighbor:
          num_neigh = len(data_no_label) # void # of neighbors >= sample size
        else:
          num_neigh = self.neighbor
        knn = NN(n_neighbors=num_neigh,p=self.r,algorithm='ball_tree').fit(data_no_label)
        if self.auto:
          to_add = to_add
        else:
          to_add = self.up_to_num
        for _ in range(to_add):
          rand_ngbr, sample = get_ngbr(data_no_label, knn)
          new_row = []
          for i, one in enumerate(rand_ngbr):
            gap = random.random()
            new_row.append(max(0, sample[i] + (
            sample[i] - one) * gap))  # here, feature vlaue should >=0
          new_row.append(label)
          total_data.append(new_row)
    return pd.DataFrame(total_data)

In [ ]:
def apply_smote(df):
    df.reset_index(drop=True,inplace=True)
    cols = df.columns
    smt = smote(df)
    df = smt.run()
    df.columns = cols
    return df

In [ ]:
dataset_orig_train, dataset_orig_test = train_test_split(compas_encoded, test_size=0.2, shuffle = True)

In [ ]:
X_train, y_train = dataset_orig_train.loc[:, dataset_orig_train.columns != 'two_year_recid'], dataset_orig_train['two_year_recid']
X_test, y_test = dataset_orig_test.loc[:, dataset_orig_test.columns != 'two_year_recid'], dataset_orig_test['two_year_recid']

In [ ]:
train_df = X_train
train_df['two_year_recid'] = y_train

In [ ]:
train_df = apply_smote(train_df)

In [ ]:
y_train = train_df.two_year_recid
X_train = train_df.drop('two_year_recid', axis = 1)

#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemCinco = pd.DataFrame(resultados)
print(df_resultados_abordagemCinco)


    accuracy    recall  precision       auc        f1                model
0   0.675676  0.624242   0.651899  0.671636  0.637771        Random Forest
1   0.672211  0.633333   0.644068  0.669157  0.638655  Logistic Regression
2   0.672904  0.618182   0.649682  0.668606  0.633540    Gradient Boosting
3   0.649342  0.562121   0.630952  0.642491  0.594551              XGBoost
4   0.669439  0.598485   0.650741  0.663866  0.623520             CatBoost
5   0.667360  0.639394   0.635542  0.665163  0.637462     Ridge Classifier
6   0.664588  0.645455   0.630178  0.663085  0.637725       SGD Classifier
7   0.674290  0.630303   0.647975  0.670835  0.639017                  SVC
8   0.615385  0.533333   0.587646  0.608940  0.559174           KNeighbors
9   0.652807  0.439394   0.688836  0.636044  0.536540           GaussianNB
10  0.667360  0.639394   0.635542  0.665163  0.637462        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 1443 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 1443 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.149, mean = 0.477, max = 0.997
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.991, mean = -0.0195, max = 0.85
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data       

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 2 metrics: FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.181146  0.936798  1.096346  1.582474  1.473186
Métricas de viés detectadas para Random Forest: TPR=1.181146, ACC=0.936798, PPV=1.096346, FPR=1.582474, STP=1.473186
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR      ACC      PPV   

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 6 - ADASYN

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
compas_encoded.sex.value_counts()

,count
sex,
1,5819
0,1395


In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

# Aplicar ADASYN
print(group.value_counts())

adasyn = ADASYN(random_state=42,n_neighbors=1, sampling_strategy='minority')
X_train_adasyn, y_group = adasyn.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_adasyn['sex'] = y_split[0].astype(int)
y_income = y_split[1].astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_adasyn['sex'].value_counts())


1_0    2119
1_1    1932
0_0     637
0_1     361
Name: count, dtype: int64
1
1    4120
0    2756
Name: count, dtype: int64
sex
1    4051
0    2825
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_adasyn, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.633718  0.687891   0.571552  0.639306  0.624349        Random Forest
1   0.622633  0.634656   0.565581  0.623873  0.598131  Logistic Regression
2   0.626790  0.707724   0.562189  0.635138  0.626617    Gradient Boosting
3   0.637413  0.691023   0.575152  0.642943  0.627786              XGBoost
4   0.636952  0.676409   0.576512  0.641021  0.622478             CatBoost
5   0.623095  0.639875   0.565498  0.624826  0.600392     Ridge Classifier
6   0.625404  0.743215   0.557557  0.637556  0.637136       SGD Classifier
7   0.627252  0.685804   0.564918  0.633291  0.619519                  SVC
8   0.640647  0.585595   0.595541  0.634968  0.590526           KNeighbors
9   0.561663  0.343424   0.506934  0.539152  0.409459           GaussianNB
10  0.628176  0.722338   0.562145  0.637888  0.632252        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.00792, mean = 0.519, max = 0.991
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.974, mean = -0.0767, max = 0.873
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 5 metrics: TPR, ACC, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  0.678609  1.739241  1.871508  0.308036  0.489059
Métricas de viés detectadas para Random Forest: TPR=0.678609, ACC=1.739241, PPV=1.871508, FPR=0.308036, STP=0.489059
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 5 metrics: TPR, ACC, PPV, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 7 - Random Oversampling

In [ ]:
from imblearn.over_sampling import RandomOverSampler

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

# Agora balanceia os grupos combinados (ex: mulher_<=50K, homem_>50K, etc.)
ros = RandomOverSampler(random_state=42)
X_train_ros, y_group = ros.fit_resample(X_train, group)

# Separar novamente income e sex
#X_train_ros['sex'] = X_train_ros['sex']  # já está incluso
y_split = y_group.str.split('_', expand=True)
X_train_ros['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)            # target original: income

# Verifica resultado
print(y_income.value_counts())
print(X_train_ros['sex'].value_counts())


1
1    4238
0    4238
Name: count, dtype: int64
sex
1    4238
0    4238
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_ros, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.682679  0.651357   0.638690  0.679448  0.644961        Random Forest
1   0.680370  0.621086   0.643939  0.674254  0.632306  Logistic Regression
2   0.679908  0.670146   0.630029  0.678901  0.649469    Gradient Boosting
3   0.676212  0.639875   0.632611  0.672464  0.636222              XGBoost
4   0.681755  0.649269   0.637949  0.678404  0.643559             CatBoost
5   0.682217  0.624217   0.645788  0.676234  0.634820     Ridge Classifier
6   0.640185  0.805846   0.565568  0.657272  0.664658       SGD Classifier
7   0.687760  0.671190   0.640438  0.686051  0.655454                  SVC
8   0.644342  0.579332   0.601952  0.637636  0.590426           KNeighbors
9   0.658661  0.391441   0.706215  0.631097  0.503694           GaussianNB
10  0.684065  0.646138   0.642116  0.680153  0.644121        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0206, mean = 0.491, max = 0.998
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.934, mean = -0.0482, max = 0.86
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data      

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)


Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, PPV, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
       TPR       ACC       PPV       FPR  STP
1  1.34127  1.037764  1.289628  1.192913  1.4
Métricas de viés detectadas para Random Forest: TPR=1.34127, ACC=1.037764, PPV=1.289628, FPR=1.192913, STP=1.4
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 1 metric: PPV

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 8 -  NearMiss (remove exemplos da classe majoritária de forma informada, ou seja, não aleatoriamente, mas baseada na distância entre os exemplos da classe majoritária e os da classe minoritária.)

In [ ]:
from imblearn.under_sampling import NearMiss

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

near = NearMiss()
X_train_near, y_group = near.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_near['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_near['sex'].value_counts())


1
0    722
1    722
Name: count, dtype: int64
sex
0    722
1    722
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_near, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


    accuracy    recall  precision       auc        f1                model
0   0.584296  0.695198   0.522763  0.595735  0.596774        Random Forest
1   0.636028  0.687891   0.574042  0.641377  0.625831  Logistic Regression
2   0.588915  0.648225   0.528961  0.595032  0.582552    Gradient Boosting
3   0.559815  0.703549   0.501862  0.574641  0.585832              XGBoost
4   0.578753  0.704593   0.517638  0.591733  0.596817             CatBoost
5   0.648037  0.680585   0.588448  0.651394  0.631171     Ridge Classifier
6   0.648499  0.610647   0.601233  0.644595  0.605904       SGD Classifier
7   0.643880  0.729645   0.577209  0.652726  0.644537                  SVC
8   0.576905  0.626305   0.518135  0.582001  0.567108           KNeighbors
9   0.606005  0.557411   0.554517  0.600992  0.555960           GaussianNB
10  0.640647  0.706681   0.576661  0.647458  0.635084        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.168, mean = 0.569, max = 1.0
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -1.0, mean = -0.126, max = 0.832
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data           

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.316271  0.973199  1.237327  1.429708  1.428899
Métricas de viés detectadas para Random Forest: TPR=1.316271, ACC=0.973199, PPV=1.237327, FPR=1.429708, STP=1.428899
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC      

In [ ]:
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 9 - BorderlineSMOTE (Para superar a deficiência do SMOTE, ele identifica dois conjuntos de pontos — Ruído e Fronteira)

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE

In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

# Cria o rótulo combinado para balanceamento
group = X_train['sex'].astype(str) + "_" + y_train.astype(str)

border = BorderlineSMOTE(random_state = 42, kind = 'borderline-2')
X_train_border, y_group = border.fit_resample(X_train, group)

# Separar novamente income e sex
y_split = y_group.str.split('_', expand=True)
X_train_border['sex'] = y_split[0].astype(float).astype(int)
y_income = y_split[1].astype(float).astype(int)

# Verifica resultado
print(y_income.value_counts())
print(X_train_border['sex'].value_counts())


1
1    4238
0    4238
Name: count, dtype: int64
sex
1    4238
0    4238
Name: count, dtype: int64


#### Treinando para vários algoritmos

In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train_border, y_income)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier
Treinado SVC
Treinado KNeighbors
Treinado GaussianNB
Treinado MLPClassifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)

    accuracy    recall  precision       auc        f1                model
0   0.676212  0.667015   0.625857  0.675264  0.645781        Random Forest
1   0.668360  0.719207   0.605448  0.673605  0.657443  Logistic Regression
2   0.689145  0.692067   0.636888  0.689447  0.663332    Gradient Boosting
3   0.669284  0.657620   0.618861  0.668081  0.637652              XGBoost
4   0.673903  0.644050   0.628310  0.670824  0.636082             CatBoost
5   0.663741  0.729645   0.598459  0.670539  0.657573     Ridge Classifier
6   0.683141  0.670146   0.634387  0.681800  0.651777       SGD Classifier
7   0.677598  0.704593   0.619266  0.680383  0.659180                  SVC
8   0.638799  0.592902   0.591667  0.634065  0.592284           KNeighbors
9   0.662818  0.406054   0.707273  0.636333  0.515915           GaussianNB
10  0.661432  0.767223   0.590361  0.672344  0.667272        MLPClassifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.00606, mean = 0.499, max = 0.987
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.964, mean = -0.0564, max = 0.986
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data    

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier
Model fairness criado para SVC
Model fairness criado para KNeighbors
Model fairness criado para GaussianNB
Model fairness criado para MLPClassifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 1 metric: PPV

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot be automatically approved based on these metrics.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.102773  1.080315  1.377358  0.864407  1.079007
Métricas de viés detectadas para Random Forest: TPR=1.102773, ACC=1.080315, PPV=1.377358, FPR=0.864407, STP=1.079007
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 1 metric: PPV

Conclusion: your model cannot be called fair because 1 criterion exceeded acceptable limits set by epsilon.
It does not mean that your model is unfair but it cannot

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



### Abordagem 10 - Reweighting ( Reponderação sensível de classe) - Sample Weight


In [ ]:
# Dividir os dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(compas_encoded.drop(["two_year_recid"], axis=1),
                                                    compas_encoded.two_year_recid,
                                                    test_size=0.3,
                                                    random_state=42)

In [ ]:
sex_train = X_train['sex']  # ou como for sua variável

# Rebalancear o sexo no treino usando sample_weight
sex_counts = sex_train.value_counts(normalize=True)
sex_weights = {sex: 1.0 / freq for sex, freq in sex_counts.items()}

# Cria o vetor de pesos
sample_weight = sex_train.map(sex_weights).values

#### Treinando para vários algoritmos

In [ ]:
model_pipelines = {
    'Random Forest': clf_forest,
    'Logistic Regression': clf_logreg,
    'Gradient Boosting': clf_gb,
    'XGBoost': clf_xgb,
    #'LightGBM': clf_lgbm,
    'CatBoost': clf_catboost,
    'Ridge Classifier': clf_ridge,
    'SGD Classifier': clf_sgd,
    #'SVC': clf_svc,
    #'KNeighbors': clf_knn,
    #'GaussianNB': clf_nb,
    #'MLPClassifier': clf_mlp
}

In [ ]:
trained_models = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train)
    trained_models[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


In [ ]:
trained_models_balanceamento = {}

for model_name, model in model_pipelines.items():
    model.fit(X_train, y_train, classifier__sample_weight=sample_weight)
    trained_models_balanceamento[model_name] = model
    print(f"Treinado {model_name}")

Treinado Random Forest
Treinado Logistic Regression
Treinado Gradient Boosting
Treinado XGBoost
Treinado CatBoost
Treinado Ridge Classifier
Treinado SGD Classifier


#### Avaliando para as métricas comuns de avaliação

In [ ]:
resultados = []
for model_name, model in trained_models_balanceamento.items():
    y_pred = model.predict(X_test)

    # Calculando as métricas
    accuracy = accuracy_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)

    # Para AUC, você precisa das probabilidades, não das classes diretamente
    fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fpr, tpr)
    f1 = f1_score(y_test, y_pred)

    relatorio = {'accuracy': accuracy, 'recall': recall, 'precision': precision, 'auc': auc, 'f1': f1, 'model': model_name}
    resultados.append(relatorio)

# Criando um DataFrame
df_resultados_abordagemSeis = pd.DataFrame(resultados)
print(df_resultados_abordagemSeis)


   accuracy    recall  precision       auc        f1                model
0  0.688222  0.615866   0.657748  0.680758  0.636119        Random Forest
1  0.679908  0.516701   0.682759  0.663073  0.588235  Logistic Regression
2  0.686836  0.609603   0.657658  0.678870  0.632719    Gradient Boosting
3  0.679908  0.600209   0.649718  0.671687  0.623983              XGBoost
4  0.686836  0.606472   0.658730  0.678547  0.631522             CatBoost
5  0.679908  0.478079   0.703533  0.659089  0.569298     Ridge Classifier
6  0.675751  0.490605   0.687135  0.656653  0.572473       SGD Classifier


#### Avaliando fairness para o atributo protegido sex e medindo o preconceito Com dalex

In [ ]:
explainers_balanceamento = {}

for model_name, model in trained_models_balanceamento.items():
    explainer = dx.Explainer(model, X_test, y_test)
    explainers_balanceamento[model_name] = explainer
    print(f"Explainer criado para {model_name}")

Preparation of a new explainer is initiated

  -> data              : 2165 rows 9 cols
  -> target variable   : Parameter 'y' was a pandas.Series. Converted to a numpy.ndarray.
  -> target variable   : 2165 values
  -> model_class       : sklearn.ensemble._forest.RandomForestClassifier (default)
  -> label             : Not specified, model's class short name will be used. (default)
  -> predict function  : <function yhat_proba_default at 0x7912a4f27ce0> will be used (default)
  -> predict function  : Accepts only pandas.DataFrame, numpy.ndarray causes problems.
  -> predicted values  : min = 0.0288, mean = 0.457, max = 0.979
  -> model type        : classification will be used (default)
  -> residual function : difference between y and yhat (default)
  -> residuals         : min = -0.915, mean = -0.0148, max = 0.897
  -> model_info        : package sklearn

A new explainer has been created!
Explainer criado para Random Forest
Preparation of a new explainer is initiated

  -> data     

In [ ]:
performance_results_balanceamento = []

# Avaliar o desempenho de cada explicador e armazenar o resultado
for model_name, explainer in explainers_balanceamento.items():
    perf_result = explainer.model_performance().result
    perf_result['model'] = model_name  # Adicionar o nome do modelo para referência
    performance_results_balanceamento.append(perf_result)

# Concatenar todos os resultados em um único DataFrame
all_performance_results_balanceamento = pd.concat(performance_results_balanceamento, ignore_index=True)

In [ ]:
protected = X_test.sex

In [ ]:
# Dicionário para armazenar os resultados de model_fairness
model_fairness_results_balanceamento = {}

# Aplicar model_fairness a cada explicador
for model_name, explainer in explainers_balanceamento.items():
    fairness_result_balanceamento = explainer.model_fairness(protected=protected, privileged=0)
    model_fairness_results_balanceamento[model_name] = fairness_result_balanceamento

    print(f"Model fairness criado para {model_name}")

Model fairness criado para Random Forest
Model fairness criado para Logistic Regression
Model fairness criado para Gradient Boosting
Model fairness criado para XGBoost
Model fairness criado para CatBoost
Model fairness criado para Ridge Classifier
Model fairness criado para SGD Classifier


In [ ]:
# Lista para armazenar os resultados de fairness_check
fairness_checks_balanceamento = []

# Aplicar fairness_check a cada resultado de model_fairness
for model_name, fairness_result_balanceamento in model_fairness_results_balanceamento.items():
    print(f"Aplicando fairness_check para o modelo: {model_name}")
    if fairness_result_balanceamento is not None:
        # Redirecionar a saída para um buffer de string
        old_stdout = sys.stdout
        sys.stdout = mystdout = StringIO()
        try:
            fairness_result_balanceamento.fairness_check()  # Chama a função que imprime a saída
            check_result = mystdout.getvalue().strip()  # Obtém o valor do buffer de string e remove espaços em branco
        finally:
            sys.stdout = old_stdout  # Restaura a saída padrão

        # Extrair o número de métricas de viés detectadas
        print(f"Resultado do fairness_check para {model_name}: {check_result}")
        num_biases = None
        num_biases_match = re.search(r'Bias detected in (\d+) metrics', check_result)
        if num_biases_match:
            num_biases = int(num_biases_match.group(1))

        # Extrair as métricas TPR, ACC, PPV, FPR, STP
        metrics_match = re.findall(r'(\d+\.\d+)', check_result)
        if metrics_match and len(metrics_match) >= 5:
            tpr = float(metrics_match[3])
            acc = float(metrics_match[4])
            ppv = float(metrics_match[5])
            fpr = float(metrics_match[6])
            stp = float(metrics_match[7])
        else:
            tpr, acc, ppv, fpr, stp = None, None, None, None, None

        fairness_checks_balanceamento.append({
            'Modelo': model_name,
            'Num Biases Detected': num_biases,
            'TPR': tpr,
            'ACC': acc,
            'PPV': ppv,
            'FPR': fpr,
            'STP': stp
        })
        print(f"Métricas de viés detectadas para {model_name}: TPR={tpr}, ACC={acc}, PPV={ppv}, FPR={fpr}, STP={stp}")
    else:
        print(f"Resultado do fairness_check é None para o modelo {model_name}")

# Criar DataFrame com todos os resultados de fairness_check
all_fairness_checks_balanceamento = pd.DataFrame(fairness_checks_balanceamento)

# Exibir o DataFrame com os resultados
print(all_fairness_checks_balanceamento)

Aplicando fairness_check para o modelo: Random Forest
Resultado do fairness_check para Random Forest: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC       PPV       FPR       STP
1  1.589242  0.988489  1.144082  1.831169  1.871901
Métricas de viés detectadas para Random Forest: TPR=1.589242, ACC=0.988489, PPV=1.144082, FPR=1.831169, STP=1.871901
Aplicando fairness_check para o modelo: Logistic Regression
Resultado do fairness_check para Logistic Regression: Bias detected in 3 metrics: TPR, FPR, STP

Conclusion: your model is not fair because 2 or more criteria exceeded acceptable limits set by epsilon.

Ratios of metrics, based on '0'. Parameter 'epsilon' was set to 0.8 and therefore metrics should be within (0.8, 1.25)
        TPR       ACC      

In [ ]:
# Transformando o DataFrame para formato longo para facilitar a visualização
fairness_melted = all_fairness_checks_balanceamento.melt(id_vars='Modelo',
                                                              value_vars=['TPR', 'ACC', 'PPV', 'FPR', 'STP'],
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica de viés usando Plotly
fig = px.bar(
    fairness_melted,
    x='Modelo',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Justiça (TPR, ACC, PPV, FPR, STP) por Modelo",
    color_continuous_scale="coolwarm",
    labels={"Valor": "Valor da Métrica", "Modelo": "Modelo", "Métrica": "Métrica de Justiça"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Justiça",
    title_x=0.5  # Centraliza o título
)

# Mostrando o gráfico
fig.show()


In [ ]:
import matplotlib.pyplot as plt

for model_name, fairness_result in model_fairness_results_balanceamento.items():
    print(f"Gerando gráfico para {model_name}")
    fairness_result.plot()
    plt.show()


Gerando gráfico para Random Forest


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Logistic Regression


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para Gradient Boosting


...

/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para GaussianNB


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



Gerando gráfico para MLPClassifier


/usr/local/lib/python3.11/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



# Resultados Finais

In [ ]:
all_performance_results.to_excel(r"Performance dos algoritmos sem balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento.to_excel(r"Performance dos algoritmos com 1º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_dois.to_excel(r"Performance dos algoritmos com 2º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_tres.to_excel(r"Performance dos algoritmos com 3º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_performance_results_balanceamento_quatro.to_excel(r"Performance dos algoritmos com 4º abordagem de balanceamento.xlsx", index=False)

In [ ]:
all_fairness_checks.to_excel(r"Resultado da avaliação se os modelos são justos - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento.to_excel(r"Resultado da avaliação se os modelos são justos com 1º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_dois.to_excel(r"Resultado da avaliação se os modelos são justos com 2º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_tres.to_excel(r"Resultado da avaliação se os modelos são justos com 3º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

In [ ]:
all_fairness_checks_balanceamento_quatro.to_excel(r"Resultado da avaliação se os modelos são justos com 4º abordagem de balanceamento - Através da biblioteca Dalex.xlsx", index=False)

Essa comparação na verdade não deve existir, porque o resultado deve-se ser igual nas duas bibliotecas.

### abordagem dois

stastical parity = 0.44 para a biblioteca dalex | 0.20 para a biblioteca AIF360

A métrica de Statistical Parity (ou Statistical Parity Difference) avalia a igualdade de tratamento entre grupos em um modelo.
Interpretação dos valores
0.44 (Dalex): Esse valor sugere uma diferença de 44% na taxa de resultados favoráveis entre os grupos. Isso indica um certo nível de viés, com o modelo favorecendo mais um grupo do que o outro.

0.20 (AIF360): Um valor de 0.20 representa uma diferença de 20% entre os grupos, o que ainda indica alguma disparidade, mas menor que a de 0.44.

O valor de 0.20 da biblioteca AIF360 seria o melhor, pois está mais próximo de zero, indicando uma menor disparidade entre os grupos e, portanto, menos viés. Em geral, quanto mais próximo de zero, melhor é o resultado em termos de equidade.

In [ ]:
clf_xgb_dalex.plot()

/usr/local/lib/python3.10/dist-packages/dalex/fairness/_group_fairness/utils.py:209: FutureWarning:

The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.



In [ ]:
# Transformando o DataFrame para formato longo
fairness_melted = all_fairness_checks_balanceamento_dois_aif360.melt(id_vars='model',
                                                              var_name='Métrica',
                                                              value_name='Valor')

# Plotando o gráfico de barras para cada métrica usando Plotly
fig = px.bar(
    fairness_melted,
    x='model',
    y='Valor',
    color='Métrica',
    title="Comparação de Métricas de Viés entre Modelos",
    labels={"model": "Modelo", "Valor": "Valor da Métrica", "Métrica": "Métrica de Viés"}
)

# Ajustando layout para melhorar a legibilidade
fig.update_layout(
    xaxis_tickangle=-45,
    legend_title_text="Métrica de Viés",
    title_x=0.5  # Centraliza o título
)